In [1]:
# импорты, которые точно понадобятся
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Данные взяты отсюда - http://jmcauley.ucsd.edu/data/amazon/
# http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz
JSON_DATA_PATH = "Video_Games_5.json"
N = 10

In [3]:
import json

def iter_json_data(path):
    with open(path) as f:
        for line in f:
            data = json.loads(line)
            yield data
            
def get_data_frame():
    uid_to_id = {}
    iid_to_id = {}
    
    cols = ["uid", "iid", "review", "rating", "dt"]
    rows = []
    for d in iter_json_data(JSON_DATA_PATH):
        uid = uid_to_id.setdefault(d["reviewerID"], len(uid_to_id))
        iid = iid_to_id.setdefault(d["asin"], len(iid_to_id))
        review = d["reviewText"]
        rating = float(d["overall"])
        dt = int(d["unixReviewTime"])
        rows.append((uid, iid, review, rating, dt))
        
    return pd.DataFrame(rows, columns=cols)

In [4]:
df = get_data_frame()
df.head()

,uid,iid,review,rating,dt
0,0,0,Installing the game was a struggle (because of...,1.0,1341792000
1,1,0,If you like rally cars get this game you will ...,4.0,1372550400
2,2,0,1st shipment received a book instead of the ga...,1.0,1403913600
3,3,0,"I got this version instead of the PS3 version,...",3.0,1315958400
4,4,0,I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,1308009600


## Готовим выборки

In [5]:
def split_df_by_dt(df, p=0.8):
    """Функция разбивает df на тестовую и тренировочную выборки по времени 
    публикации отзывов (значение времени в поле dt)
    
    :param p: персентиль значений dt, которые образуют тренировочную выборку. Например p=0.8 означает, что в 
    тренировочной части будут отзывы, соответствующие первым 80% временного интервала 
    :return: два pd.DataFrame объекта
    """
    border_dt = df.dt.quantile(p)
    print("Min=%s, border=%s, max=%s" % (df.dt.min(), border_dt, df.dt.max()))
    training_df, test_df  = df[df.dt <= border_dt], df[df.dt > border_dt]
    print("Размер до очистки:", training_df.shape, test_df.shape)
    # удаляем из тестовых данных строки, соответствующие пользователям или объектам, 
    # которых нет в тренировочных данных 
    # (пользователи - избегаем проблем для персональных систем, объекты - для всех)
    test_df = test_df[test_df.uid.isin(training_df.uid) & test_df.iid.isin(training_df.iid)]
    print("Размер после очистки:", training_df.shape, test_df.shape)
    return training_df, test_df

In [6]:
training_df, test_df = split_df_by_dt(df)
del df

Min=939859200, border=1377129600.0, max=1405987200
Размер до очистки: (185427, 5) (46353, 5)
Размер после очистки: (185427, 5) (19174, 5)


In [7]:
def hit_ratio(recs_dict, test_dict):
    """Функция считает метрику hit-ration для двух словарей
    :recs_dict: словарь рекомендаций типа {uid: {iid: score, ...}, ...}
    :test_dict: тестовый словарь типа {uid: {iid: score, ...}, ...}
    """
    hits = 0
    for uid in test_dict:
        if set(test_dict[uid].keys()).intersection(recs_dict.get(uid, {})):
            hits += 1
    return hits / len(test_dict)

In [8]:
def get_test_dict(test_df):
    """Функция, конвертирующая тестовый df в словарь
    """
    test_dict = {}
    for t in test_df.itertuples():
        test_dict.setdefault(t.uid, {})
        test_dict[t.uid][t.iid] = t.rating
    return test_dict

test_dict = get_test_dict(test_df)

## Non-personalized RS

In [9]:
class BasicRecommender(object):
    def __init__(self):
        pass
    
    def get_recs(self, uid, top):
        """Строит рекомендации для пользователя uid
        :return: словарь типа {iid: score, ...}
        """
        return {}
    
    def get_batch_recs(self, uids, top):
        """Строит рекомендации для нескольких пользователей uids
        :return: словарь типа {uid: {iid: score, ...}, ...}
        """
        return {uid: self.get_recs(uid, top) for uid in uids}
    
class NonPersRecommender(BasicRecommender):
    def __init__(self, df):
        super(NonPersRecommender, self).__init__()
        self.recs = self._prepare_recs(df)
        
    def _prepare_recs(self, df):
        return pd.Series([])
    
    def get_recs(self, uid, top):
        return self.recs[:top].to_dict()
    
    def get_batch_recs(self, uids, top):
        non_pers_recs = self.get_recs(None, top)
        return {uid: non_pers_recs for uid in uids}

In [10]:
class MostReviewedRS(NonPersRecommender):
    def _prepare_recs(self, df):
        # считаем количество отзывов для каждого объекта (pandas сортирует их по убыванию)
        return df.iid.value_counts()  

Попробуем предлагать только наиболее актуальные игры, наиболее обсуждаемы за последние днию Количество дней подберем на кросс-валидации.

In [11]:
train_val_0, test_val_0 = split_df_by_dt(training_df)
train_val_1, test_val_1 = split_df_by_dt(train_val_0)
train_val_2, test_val_2 = split_df_by_dt(train_val_1)
train_val_3, test_val_3 = split_df_by_dt(train_val_2)

train_val = [train_val_0, train_val_1, train_val_2, train_val_3]
test_val = [test_val_0, test_val_1, test_val_2, test_val_3]

Min=939859200, border=1355097600.0, max=1377129600
Размер до очистки: (148429, 5) (36998, 5)
Размер после очистки: (148429, 5) (15356, 5)
Min=939859200, border=1320883200.0, max=1355097600
Размер до очистки: (118800, 5) (29629, 5)
Размер после очистки: (118800, 5) (9174, 5)
Min=939859200, border=1288051200.0, max=1320883200
Размер до очистки: (95090, 5) (23710, 5)
Размер после очистки: (95090, 5) (6928, 5)
Min=939859200, border=1253577600.0, max=1288051200
Размер до очистки: (76101, 5) (18989, 5)
Размер после очистки: (76101, 5) (4898, 5)


In [12]:
def Validation(train_val, test_val, param_grid):
    best_hr = 0
    best_param = ''
    for param in param_grid:
        for train_df, test_df in zip(train_val, test_val):
            fold_hr =[]
            recs_df = train_df.loc[train_df['dt']>=(train_df['dt'].max()-86400*param)]
            recommender = MostReviewedRS(df=recs_df)
            recs = recommender.get_batch_recs(top=10,uids=training_df.uid.unique())
            test_dict = get_test_dict(test_df)
            metric = hit_ratio(recs_dict=recs, test_dict=test_dict)
            fold_hr.append(metric)
        avg_fold_hr = np.mean(fold_hr)
        if avg_fold_hr > best_hr:
            best_hr = avg_fold_hr
            best_param = param               
    return best_param, best_hr

In [13]:
params = range(25)
Validation(train_val, test_val, params)

(13, 0.12171052631578948)

In [14]:
days=13

pred_df = training_df.loc[training_df['dt']>=(training_df['dt'].max()-86400*days)]

recommender = MostReviewedRS(df=pred_df)
recs = recommender.get_batch_recs(top=10,uids=training_df.uid.unique())

hit_ratio(recs_dict=recs, test_dict=test_dict)

0.0748349229640499